In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/clean_raw_data.csv
/kaggle/input/clean_data_mean.csv
/kaggle/input/raw_data.csv
/kaggle/input/clean_data_GCN.csv
/kaggle/input/FillZero_minmax_baseline/clean_data_minmax_fill-zero.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/val/val_week1_2.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/test/test_week2.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/train/clean_data_week2.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/train/5-folds/data_part_2.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/train/5-folds/data_part_3.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/train/5-folds/data_part_4.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/train/5-folds/data_part_1.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/train/5-folds/data_part_5.csv
/kaggle/input/FillZero_minmax_baseline/clean_week3/val/val_week1_2_3.csv
/kaggle/input/FillZero_minmax_baseline/clean_week3/test/test_week3.csv
/kaggle/input/FillZero_minmax_bas

In [2]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from keras_tuner import RandomSearch
from sklearn.model_selection import StratifiedKFold
import time
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_recall_fscore_support, roc_auc_score

In [3]:
# Biến global cho base path
BASE_PATH = "/kaggle/input/Median_minmax_baseline_version2"
# Tuần và số phần fold
weeks = ['week1', 'week2', 'week3', 'week4']
fold_parts = 5

# Tạo five_fold_files
five_fold_files = {
    week: [
        f"{BASE_PATH}/clean_{week}/train/5-folds/data_part_{i}.csv"
        for i in range(1, fold_parts + 1)
    ]
    for week in weeks
}

# Tạo file_validation
file_validation = {
    'week1': [f"{BASE_PATH}/clean_week1/val/val_week1.csv"],
    'week2': [f"{BASE_PATH}/clean_week2/val/val_week1_2.csv"],
    'week3': [f"{BASE_PATH}/clean_week3/val/val_week1_2_3.csv"],
    'week4': [f"{BASE_PATH}/clean_week4/val/val_week1_2_3_4.csv"]
}

# Tạo file_test
file_test = {
    week: [f"{BASE_PATH}/clean_{week}/test/test_{week}.csv"]
    for week in weeks
}


## Tìm siêu tham số tốt nhất cho từng tuần

In [4]:
# Định nghĩa Focal Loss
def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        y_pred = K.clip(y_pred, K.epsilon(), 1. - K.epsilon())
        cross_entropy = -y_true * K.log(y_pred)
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy
        return K.sum(loss, axis=-1)
    return focal_loss_fixed

# Tạo hàm train cho từng tuần
def train_week_model(week_number, file_paths_train, file_validataion):
    # Đọc dữ liệu
    train_data = pd.read_csv(file_paths_train)
    val_data = pd.read_csv(file_validataion)
    
    # Tách đặc trưng và nhãn
    X_train = train_data.drop(columns=["classification_encoded", "user_id", "course_id", "school", "enroll_time", "classification"])
    y_train = train_data["classification_encoded"]

    X_val = val_data.drop(columns=["classification_encoded", "user_id", "course_id", "school", "enroll_time", "classification"])
    y_val = val_data["classification_encoded"]
    
    # Áp dụng Over-sampling cho dữ liệu huấn luyện bằng SMOTE
    oversampler = SMOTE(sampling_strategy='auto', random_state=42)
    X_train_res, y_train_res = oversampler.fit_resample(X_train, y_train)
    
    # Reshape dữ liệu cho mô hình BiLSTM
    X_train_res = X_train_res.values.reshape(X_train_res.shape[0], X_train_res.shape[1], 1)
    X_val = X_val.values.reshape(X_val.shape[0], X_val.shape[1], 1)
    
    # One-hot encode nhãn
    y_train_res = tf.keras.utils.to_categorical(y_train_res, num_classes=5)
    y_val = tf.keras.utils.to_categorical(y_val, num_classes=5)
    
    def build_model(hp):
        inputs = tf.keras.Input(shape=(X_train_res.shape[1], 1))  # Khởi tạo đầu vào
        
        # GRU layer 1
        x = layers.GRU(
            units=hp.Int('units_1', min_value=32, max_value=256, step=32),
            return_sequences=True
        )(inputs)
        x = layers.Dropout(rate=hp.Float('dropout_1', min_value=0.1, max_value=0.5, step=0.1))(x)
        
        # GRU layer 2
        x = layers.GRU(
            units=hp.Int('units_2', min_value=32, max_value=256, step=32),
            return_sequences=False
        )(x)
        x = layers.Dropout(rate=hp.Float('dropout_2', min_value=0.1, max_value=0.5, step=0.1))(x)
        
        # Lớp đầu ra
        outputs = layers.Dense(5, activation='softmax')(x)
        
        # Khởi tạo mô hình
        model = tf.keras.Model(inputs=inputs, outputs=outputs)
        
        # Compile với Focal Loss
        model.compile(optimizer=tf.keras.optimizers.Adam(
                          learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
                      loss=focal_loss(gamma=2., alpha=0.25),
                      metrics=['accuracy'])
        
        return model

    
    # Khởi tạo RandomSearch tuner
    tuner = RandomSearch(
        build_model,
        objective='val_accuracy',
        max_trials=10,
        executions_per_trial=1,
        directory='my_dir',
        project_name=f'bilstm_tuning_week{week_number}'
    )
    
    # Tìm kiếm siêu tham số tốt nhất
    tuner.search(X_train_res, y_train_res,
                 epochs=20,
                 validation_data=(X_val, y_val),
                 batch_size=32)
    
    # Trả về kết quả tối ưu cho tuần
    best_params = tuner.get_best_hyperparameters(num_trials=1)[0]
    return best_params

In [5]:
# Định nghĩa đường dẫn đến dữ liệu cho từng tuần
file_paths_train = {
    week: f"{BASE_PATH}/clean_{week}/train/clean_data_{week}.csv"
    for week in weeks
}

# Định nghĩa file_validation theo quy luật riêng
file_validation = {
    f"week{idx + 1}": f"{BASE_PATH}/clean_week{idx + 1}/val/val_week{'_'.join(str(i) for i in range(1, idx + 2))}.csv"
    for idx in range(len(weeks))
}

In [6]:
# Tìm tham số tốt nhất cho từng tuần
best_params_week1 = train_week_model(1, file_paths_train["week1"], file_validation["week1"])
best_params_week2 = train_week_model(2, file_paths_train["week2"], file_validation["week2"])
best_params_week3 = train_week_model(3, file_paths_train["week3"], file_validation["week3"])
best_params_week4 = train_week_model(4, file_paths_train["week4"], file_validation["week4"])

# In thông tin chi tiết các tham số tối ưu
print("Best Parameters for Week 1:")
for param_name in best_params_week1.values.keys():
    print(f"{param_name}: {best_params_week1.get(param_name)}")

print("\nBest Parameters for Week 2:")
for param_name in best_params_week2.values.keys():
    print(f"{param_name}: {best_params_week2.get(param_name)}")

print("\nBest Parameters for Week 3:")
for param_name in best_params_week3.values.keys():
    print(f"{param_name}: {best_params_week3.get(param_name)}")

print("\nBest Parameters for Week 4:")
for param_name in best_params_week4.values.keys():
    print(f"{param_name}: {best_params_week4.get(param_name)}")


Trial 10 Complete [00h 02m 33s]
val_accuracy: 0.9896278381347656

Best val_accuracy So Far: 0.9993898868560791
Total elapsed time: 00h 25m 50s
Best Parameters for Week 1:
units_1: 128
dropout_1: 0.1
units_2: 224
dropout_2: 0.4
learning_rate: 0.0069108517839247645

Best Parameters for Week 2:
units_1: 224
dropout_1: 0.4
units_2: 32
dropout_2: 0.2
learning_rate: 0.0016017872820259641

Best Parameters for Week 3:
units_1: 64
dropout_1: 0.2
units_2: 96
dropout_2: 0.2
learning_rate: 0.0015027297638441546

Best Parameters for Week 4:
units_1: 224
dropout_1: 0.1
units_2: 64
dropout_2: 0.4
learning_rate: 0.001143448498353399


## Danh sách tham số tốt nhất của từng tuần

In [7]:
# Danh sách tham số tốt nhất
best_params = {
    "week1": best_params_week1,
    "week2": best_params_week2,
    "week3": best_params_week3,
    "week4": best_params_week4
}

In [8]:
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras import backend as K

# Định nghĩa Focal Loss
def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        y_pred = K.clip(y_pred, K.epsilon(), 1. - K.epsilon())
        cross_entropy = -y_true * K.log(y_pred)
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy
        return K.sum(loss, axis=-1)
    
    return focal_loss_fixed

# Xây dựng mô hình BiLSTM
def build_GRU_model(params, input_shape):
    inputs = tf.keras.Input(shape=input_shape)  # Định nghĩa đầu vào
    
    # GRU layer 1
    x = layers.GRU(
        units=params.get('units_1'),
        return_sequences=True
    )(inputs)
    x = layers.Dropout(rate=params.get('dropout_1', 0.2))(x)
    
    # GRU layer 2
    x = layers.GRU(
        units=params.get('units_2', 32),
        return_sequences=False
    )(x)
    x = layers.Dropout(rate=params.get('dropout_2', 0.2))(x)
    
    # Lớp đầu ra
    outputs = layers.Dense(5, activation='softmax')(x)
    
    # Khởi tạo mô hình
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    
    # Compile với Focal Loss
    model.compile(optimizer=tf.keras.optimizers.Adam(
                      learning_rate=params['learning_rate']),
                  loss=focal_loss(gamma=params.get('gamma', 2.), alpha=params.get('alpha', 0.25)),
                  metrics=['accuracy'])
    
    return model


In [9]:
# Biến lưu kết quả tổng quát
overall_results_5folds = []

# Lặp qua từng tuần
for week, file_paths in five_fold_files.items():
    print(f"\nProcessing {week} with best parameters...")
    params = best_params[week].values
    print(f"best parameters for {week}: {params}")
    
    # Biến lưu kết quả cho từng tuần
    week_results = {
        "week": week,
        "accuracy_per_fold": [],
        "precision_per_label": [],
        "recall_per_label": [],
        "f1_score_per_label": [],
        "auc_roc_per_label": [],    # AUC từng lớp
        "auc_roc_macro": [],        # AUC macro
        "auc_roc_weighted": [],     # AUC weighted (tự tính)
        "precision_macro": [],
        "recall_macro": [],
        "f1_macro": [],
        "precision_weighted": [],
        "recall_weighted": [],
        "f1_weighted": [],
        "confusion_matrices": [],
        "train_times": [],
        "test_times": []
    }

    # Lặp qua từng fold
    for i in range(len(file_paths)):
        print(f"Fold {i+1}: Using file {file_paths[i]} as test set")
        
        # Tải dữ liệu
        test_data = pd.read_csv(file_paths[i])
        train_data = pd.concat([pd.read_csv(file_paths[j]) for j in range(len(file_paths)) if j != i])
        
        # Tách X và y
        X_train = train_data.drop(columns=["classification_encoded", "user_id",
                                           "course_id", "school", "enroll_time", "classification"])
        y_train = to_categorical(train_data['classification_encoded'], num_classes=5)
        
        X_test = test_data.drop(columns=["classification_encoded", "user_id",
                                         "course_id", "school", "enroll_time", "classification"])
        y_test = to_categorical(test_data['classification_encoded'], num_classes=5)

        # Reshape dữ liệu cho LSTM
        X_train = X_train.to_numpy().reshape((X_train.shape[0], 1, X_train.shape[1]))
        X_test = X_test.to_numpy().reshape((X_test.shape[0], 1, X_test.shape[1]))

        # Xây dựng mô hình với tham số tốt nhất
        input_shape = (X_train.shape[1], X_train.shape[2])
        model = build_GRU_model(params, input_shape)
        
        # Bắt đầu tính thời gian huấn luyện
        start_train = time.time()
        model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test), batch_size=32)
        end_train = time.time()
        
        # Bắt đầu tính thời gian kiểm thử
        start_test = time.time()
        y_pred = model.predict(X_test)
        end_test = time.time()
        
        # Tính thời gian và lưu lại
        train_time = end_train - start_train
        test_time = end_test - start_test
        week_results["train_times"].append(train_time)
        week_results["test_times"].append(test_time)

        # Đánh giá mô hình trên tập kiểm thử của fold hiện tại
        _, accuracy = model.evaluate(X_test, y_test, verbose=0)
        week_results["accuracy_per_fold"].append(accuracy)
        
        # Dự đoán
        y_pred_classes = y_pred.argmax(axis=1)
        y_test_classes = y_test.argmax(axis=1)
        
        # Tính các chỉ số cho mỗi fold
        precision, recall, f1, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average=None)
        precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average='macro')
        precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average='weighted')
        conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)
        
        # Tính AUC-ROC
        try:
            # Tính AUC macro và theo từng lớp với OvR
            auc_macro = roc_auc_score(y_test, y_pred, multi_class="ovr", average="macro")
            auc_per_class = roc_auc_score(y_test, y_pred, multi_class="ovr", average=None)
            # Tính AUC weighted: tính trọng số theo số mẫu của từng lớp
            supports = np.bincount(y_test_classes, minlength=5)
            auc_weighted = np.sum(auc_per_class * supports) / np.sum(supports)
        except Exception as e:
            print(f"Lỗi khi tính AUC: {e}")
            auc_macro = np.nan
            auc_per_class = [np.nan] * 5
            auc_weighted = np.nan
            
        # Lưu kết quả của fold hiện tại
        week_results["precision_per_label"].append(precision)
        week_results["recall_per_label"].append(recall)
        week_results["f1_score_per_label"].append(f1)
        week_results["auc_roc_per_label"].append(auc_per_class)  # AUC từng lớp
        week_results["auc_roc_macro"].append(auc_macro)          # AUC macro
        week_results["auc_roc_weighted"].append(auc_weighted)      # AUC weighted
        week_results["confusion_matrices"].append(conf_matrix)
        week_results["precision_macro"].append(precision_macro)
        week_results["recall_macro"].append(recall_macro)
        week_results["f1_macro"].append(f1_macro)
        week_results["precision_weighted"].append(precision_weighted)
        week_results["recall_weighted"].append(recall_weighted)
        week_results["f1_weighted"].append(f1_weighted)

    # Tính trung bình cho từng nhãn
    average_precision_per_label = np.mean(week_results["precision_per_label"], axis=0)
    average_recall_per_label = np.nanmean(week_results["recall_per_label"], axis=0)
    average_f1_per_label = np.nanmean(week_results["f1_score_per_label"], axis=0)
    average_auc_per_label = np.nanmean(week_results["auc_roc_per_label"], axis=0)
    average_confusion_matrix = np.nanmean(week_results["confusion_matrices"], axis=0)
    average_train_time = sum(week_results["train_times"]) / len(week_results["train_times"])
    average_test_time = sum(week_results["test_times"]) / len(week_results["test_times"])
    average_accuracy = np.nanmean(week_results["accuracy_per_fold"])
    average_precision_macro = np.nanmean(week_results["precision_macro"])
    average_recall_macro = np.nanmean(week_results["recall_macro"])
    average_f1_macro = np.nanmean(week_results["f1_macro"])
    average_auc_macro = np.nanmean(week_results["auc_roc_macro"])
    average_precision_weighted = np.nanmean(week_results["precision_weighted"])
    average_recall_weighted = np.nanmean(week_results["recall_weighted"])
    average_f1_weighted = np.nanmean(week_results["f1_weighted"])
    average_auc_weighted = np.nanmean(week_results["auc_roc_weighted"])


    # Tạo DataFrame cho precision, recall, f1-score
    labels = np.unique(y_test_classes)  # Lấy nhãn từ y_test_classes
    metrics_df = pd.DataFrame({
        "Label": labels,
        "Average Precision": average_precision_per_label,
        "Average Recall": average_recall_per_label,
        "Average F1-Score": average_f1_per_label,
        "Average AUC": average_auc_per_label
    })
    
    # Tạo DataFrame cho confusion matrix
    confusion_df = pd.DataFrame(average_confusion_matrix, index=labels, columns=labels)
    # In kết quả Accuracy và Macro metrics
    print("\n=== Average Accuracy ===")
    print(f"{average_accuracy:.4f}")
    print("\n=== Average Macro Metrics ===")
    print(f"Macro Precision: {average_precision_macro:.4f}")
    print(f"Macro Recall: {average_recall_macro:.4f}")
    print(f"Macro F1-Score: {average_f1_macro:.4f}")
    print(f"Macro AUC-ROC: {average_auc_macro:.4f}")
    print("\n=== Average Weighted Metrics ===")
    print(f"Weighted Precision: {average_precision_weighted:.4f}")
    print(f"Weighted Recall: {average_recall_weighted:.4f}")
    print(f"Weighted F1-Score: {average_f1_weighted:.4f}")
    print(f"Weighted AUC-ROC: {average_auc_weighted:.4f}")
    print("\n=== Average Metrics per Label ===")
    print(metrics_df)
    print("\n=== Average Confusion Matrix ===")
    print(confusion_df)
    
    # Cập nhật kết quả cho tuần hiện tại
    week_results.update({
        "average_accuracy": average_accuracy,
        "average_precision_macro": average_precision_macro,
        "average_recall_macro": average_recall_macro,
        "average_f1_macro": average_f1_macro,
        "average_auc_macro": average_auc_macro,
        "average_precision_weighted": average_precision_weighted,
        "average_recall_weighted": average_recall_weighted,
        "average_f1_weighted": average_f1_weighted,
        "average_auc_weighted": average_auc_weighted,
        "average_metrics_df": metrics_df,
        "average_confusion_matrix": confusion_df,
        "average_train_times": average_train_time,
        "average_test_times": average_test_time,
    })
    overall_results_5folds.append(week_results)


Processing week1 with best parameters...
best parameters for week1: {'units_1': 128, 'dropout_1': 0.1, 'units_2': 224, 'dropout_2': 0.4, 'learning_rate': 0.0069108517839247645}
Fold 1: Using file /kaggle/input/Median_minmax_baseline_version2/clean_week1/train/5-folds/data_part_1.csv as test set
Epoch 1/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.6105 - loss: 0.1452 - val_accuracy: 0.6477 - val_loss: 0.1260
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6253 - loss: 0.1314 - val_accuracy: 0.6561 - val_loss: 0.1248
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6420 - loss: 0.1304 - val_accuracy: 0.6657 - val_loss: 0.1212
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6525 - loss: 0.1238 - val_accuracy: 0.6199 - val_loss: 0.1258
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6324 - loss: 0.1285 - val_accuracy: 0.6592 - val_loss: 0.1204
Epoch 6/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy:

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.6124 - loss: 0.1448 - val_accuracy: 0.6062 - val_loss: 0.1315
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6368 - loss: 0.1305 - val_accuracy: 0.6969 - val_loss: 0.1195
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6185 - loss: 0.1282 - val_accuracy: 0.5986 - val_loss: 0.1243
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6255 - loss: 0.1312 - val_accuracy: 0.6927 - val_loss: 0.1161
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6336 - loss: 0.1279 - val_accuracy: 0.6752 - val_loss: 0.1142
Epoch 6/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6218 - loss: 0.1297 - val_accuracy: 0.6828 - val_loss: 0.1130
Epoch 7/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6390 - loss: 0.1262 - val_accuracy: 0.6817 - val_loss: 0.1116
Epoch 8/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6500 - loss: 0.1242 - val_accuracy: 0.6214 - val_

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


328/328 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.5908 - loss: 0.1472 - val_accuracy: 0.6180 - val_loss: 0.1246
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6285 - loss: 0.1324 - val_accuracy: 0.6855 - val_loss: 0.1191
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6401 - loss: 0.1252 - val_accuracy: 0.6981 - val_loss: 0.1243
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6365 - loss: 0.1271 - val_accuracy: 0.6710 - val_loss: 0.1213
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6522 - loss: 0.1219 - val_accuracy: 0.6775 - val_loss: 0.1197
Epoch 6/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6561 - loss: 0.1198 - val_accuracy: 0.7183 - val_loss: 0.1083
Epoch 7/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6846 - loss: 0.1140 - val_accuracy: 0.7274 - val_loss: 0.1084
Epoch 8/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6748 - loss: 0.1170 - val_accuracy: 0.7156 - val_

...

328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.5955 - loss: 0.1503 - val_accuracy: 0.6247 - val_loss: 0.1190
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6325 - loss: 0.1299 - val_accuracy: 0.6342 - val_loss: 0.1216
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6275 - loss: 0.1286 - val_accuracy: 0.6293 - val_loss: 0.1293
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6228 - loss: 0.1341 - val_accuracy: 0.6587 - val_loss: 0.1189
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6416 - loss: 0.1267 - val_accuracy: 0.6838 - val_loss: 0.1185
Epoch 6/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6379 - loss: 0.1310 - val_accuracy: 0.6278 - val_loss: 0.1284
Epoch 7/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6318 - loss: 0.1347 - val_accuracy: 0.6243 - val_loss: 0.1228
Epoch 8/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6233 - loss: 0.1339 - val_accuracy: 0.6396 - val_

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.6175 - loss: 0.1464 - val_accuracy: 0.6148 - val_loss: 0.1272
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6328 - loss: 0.1342 - val_accuracy: 0.6232 - val_loss: 0.1240
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6313 - loss: 0.1306 - val_accuracy: 0.6606 - val_loss: 0.1254
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6478 - loss: 0.1258 - val_accuracy: 0.6861 - val_loss: 0.1219
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6289 - loss: 0.1311 - val_accuracy: 0.6274 - val_loss: 0.1206
Epoch 6/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6331 - loss: 0.1290 - val_accuracy: 0.6632 - val_loss: 0.1191
Epoch 7/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6343 - loss: 0.1311 - val_accuracy: 0.6007 - val_loss: 0.1283
Epoch 8/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6267 - loss: 0.1303 - val_accuracy: 0.6571 - val_

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



=== Average Accuracy ===
0.7074

=== Average Macro Metrics ===
Macro Precision: 0.4170
Macro Recall: 0.3670
Macro F1-Score: 0.3702
Macro AUC-ROC: 0.8132

=== Average Weighted Metrics ===
Weighted Precision: 0.6453
Weighted Recall: 0.7074
Weighted F1-Score: 0.6634
Weighted AUC-ROC: 0.8289

=== Average Metrics per Label ===
   Label  Average Precision  Average Recall  Average F1-Score  Average AUC
0      0           0.590909        0.545667          0.558659     0.786498
1      1           0.000000        0.000000          0.000000     0.740704
2      2           0.439852        0.194375          0.267966     0.841316
3      3           0.295846        0.180838          0.196671     0.851328
4      4           0.758197        0.913935          0.827893     0.846035

=== Average Confusion Matrix ===
       0    1     2     3       4
0  327.4  0.0   4.8  16.6   251.2
1   42.6  0.0   0.4   2.4    42.0
2   40.2  0.0  32.0   5.4    86.8
3   39.4  0.0   1.6  30.2    96.0
4  115.2  0.0   5.4  

## Kết quả cross validation trên 5-folds

In [10]:
# Duyệt qua các tuần trong overall_results
for week_result in overall_results_5folds:
    week = week_result["week"]
    average_train_time = np.mean(week_result["train_times"])
    average_test_time = np.mean(week_result["test_times"])
    average_metrics_df = week_result["average_metrics_df"]
    average_accuracy = np.mean(week_results["accuracy_per_fold"])
    average_confusion_matrix = week_result["average_confusion_matrix"]
    
    # In kết quả
    print(f"\n=== Results for {week} ===")
    print(f"Average Accurancy: {average_accuracy}")
    print(f"Average Train Time: {average_train_time:.4f} seconds")
    print(f"Average Test Time: {average_test_time:.4f} seconds")
    print(f"Average AUC Macro: {average_auc_macro}")
    print(f"Average AUC Weighted: {average_auc_weighted}")
    print("\nAverage Precision, Recall, F1-Score, AUC-ROC per Label:")
    print(average_metrics_df)
    print("\nAverage Confusion Matrix:")
    print(average_confusion_matrix)



=== Results for week1 ===
Average Accurancy: 0.9578279972076416
Average Train Time: 85.0733 seconds
Average Test Time: 0.4292 seconds
Average AUC Macro: 0.9947132314240156
Average AUC Weighted: 0.9972342206040304

Average Precision, Recall, F1-Score, AUC-ROC per Label:
   Label  Average Precision  Average Recall  Average F1-Score  Average AUC
0      0           0.590909        0.545667          0.558659     0.786498
1      1           0.000000        0.000000          0.000000     0.740704
2      2           0.439852        0.194375          0.267966     0.841316
3      3           0.295846        0.180838          0.196671     0.851328
4      4           0.758197        0.913935          0.827893     0.846035

Average Confusion Matrix:
       0    1     2     3       4
0  327.4  0.0   4.8  16.6   251.2
1   42.6  0.0   0.4   2.4    42.0
2   40.2  0.0  32.0   5.4    86.8
3   39.4  0.0   1.6  30.2    96.0
4  115.2  0.0   5.4  17.4  1465.6

=== Results for week2 ===
Average Accurancy: 0.

## Kiểm tra trên tập test

In [11]:
# Mảng lưu dữ liệu của các tuần
results = []

def process_week(week_num, best_params, results):
    print(f"\n=== Processing Week {week_num} ===")
    params = best_params[f"week{week_num}"].values
    # Đường dẫn tới dữ liệu tuần tương ứng
    train_path = f"{BASE_PATH}/clean_week{week_num}/train/clean_data_week{week_num}.csv"
    test_path = f"{BASE_PATH}/clean_week{week_num}/test/test_week{week_num}.csv"
    
    # Load dữ liệu
    train_data = pd.read_csv(train_path)
    test_data = pd.read_csv(test_path)
    
    # Tách X và y
    X_train = train_data.drop(columns=["classification_encoded", "user_id",
                                       "course_id", "school", "enroll_time", "classification"])
    y_train = train_data['classification_encoded']
    
    X_test = test_data.drop(columns=["classification_encoded", "user_id",
                                     "course_id", "school", "enroll_time", "classification"])
    y_test = test_data['classification_encoded']

    # Áp dụng SMOTE cho tập huấn luyện
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

    # Chuyển đổi nhãn sang dạng one-hot
    y_train_resampled = to_categorical(y_train_resampled, num_classes=5)
    y_test = to_categorical(y_test, num_classes=5)

    # Reshape dữ liệu cho LSTM
    X_train_resampled = X_train_resampled.to_numpy().reshape((X_train_resampled.shape[0], 1, X_train_resampled.shape[1]))
    X_test = X_test.to_numpy().reshape((X_test.shape[0], 1, X_test.shape[1]))

    # Xây dựng mô hình với tham số tốt nhất
    input_shape = (X_train_resampled.shape[1], X_train_resampled.shape[2])
    model = build_GRU_model(params, input_shape)
    
    # Huấn luyện mô hình
    start_train = time.time()
    model.fit(X_train_resampled, y_train_resampled, epochs=50, validation_split=0.1, batch_size=32)
    end_train = time.time()
    
    # Kiểm thử mô hình
    start_test = time.time()
    y_pred = model.predict(X_test)
    end_test = time.time()
    
    # Tính thời gian huấn luyện và kiểm thử
    train_time = end_train - start_train
    test_time = end_test - start_test
    
    # Đánh giá mô hình
    y_pred_classes = y_pred.argmax(axis=1)
    y_test_classes = y_test.argmax(axis=1)
    
    # Tính các chỉ số Precision, Recall, F1 cho từng lớp và macro
    precision, recall, f1, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average=None)
    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average='macro')
    precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average='weighted')
    conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)
    accuracy = accuracy_score(y_test_classes, y_pred_classes)
    
    # Tính AUC-ROC (với one-vs-rest)
    try:
        auc_macro = roc_auc_score(y_test, y_pred, multi_class="ovr", average="macro")
        auc_per_class = roc_auc_score(y_test, y_pred, multi_class="ovr", average=None)
        # Tính AUC weighted tự tính theo trọng số mẫu của từng lớp
        supports = np.bincount(y_test_classes, minlength=5)
        auc_weighted = np.sum(auc_per_class * supports) / np.sum(supports)
    except Exception as e:
        print(f"Lỗi khi tính AUC: {e}")
        auc_macro = np.nan
        auc_per_class = [np.nan] * 5
        auc_weighted = np.nan

    # Lưu kết quả vào mảng
    results.append({
        "week": week_num,
        "train_time": train_time,
        "test_time": test_time,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "accuracy": accuracy,
        "precision_macro": precision_macro,
        "recall_macro": recall_macro,
        "f1_macro": f1_macro,
        "precision_weighted": precision_weighted,
        "recall_weighted": recall_weighted,
        "f1_weighted": f1_weighted,
        "auc_macro": auc_macro,
        "auc_weighted": auc_weighted,
        "auc_per_class": auc_per_class,
        "confusion_matrix": conf_matrix
    })
    
    # In kết quả chi tiết
    print("\n=== Precision, Recall, F1-Score per Label ===")
    print(pd.DataFrame({
        "Label": np.unique(y_test_classes),
        "Precision": precision,
        "Recall": recall,
        "F1-Score": f1
    }))

    print("\n=== Macro Averages & Accuracy ===")
    print(f"Macro Precision: {precision_macro:.4f}")
    print(f"Macro Recall: {recall_macro:.4f}")
    print(f"Macro F1-Score: {f1_macro:.4f}")
    print(f"Accuracy: {accuracy:.4f}")
    
    print("\n=== Weighted Averages ===")
    print(f"Weighted Precision: {precision_weighted:.4f}")
    print(f"Weighted Recall: {recall_weighted:.4f}")
    print(f"Weighted F1-Score: {f1_weighted:.4f}")
    
    print("\n=== AUC-ROC ===")
    print(f"AUC Macro: {auc_macro:.4f}")
    print(f"AUC Weighted: {auc_weighted:.4f}")
    print(f"AUC per Label: {auc_per_class}")
    
    print("\n=== Confusion Matrix ===")
    print(pd.DataFrame(conf_matrix, index=np.unique(y_test_classes), columns=np.unique(y_test_classes)))
    
    print(f"\nTrain Time: {train_time:.2f} seconds")
    print(f"Test Time: {test_time:.2f} seconds")

In [12]:
process_week(1, best_params, results)


=== Processing Week 1 ===
Epoch 1/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.4248 - loss: 0.2132 - val_accuracy: 0.4320 - val_loss: 0.2762
Epoch 2/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.4780 - loss: 0.1944 - val_accuracy: 0.6702 - val_loss: 0.1801
Epoch 3/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.4873 - loss: 0.1922 - val_accuracy: 0.5538 - val_loss: 0.2631
Epoch 4/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.4977 - loss: 0.1875 - val_accuracy: 0.4585 - val_loss: 0.3206
Epoch 5/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.4749 - loss: 0.1923 - val_accuracy: 0.4515 - val_loss: 0.2700
Epoch 6/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.4867 - loss: 0.1876 - val_accuracy: 0.6223 - val_loss: 0.2116
Epoch 7/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.4894 - loss: 0.1859 - val_accuracy: 0.4590 - val_loss: 0.2723
Epoch 8/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accurac

In [13]:
process_week(2, best_params, results)


=== Processing Week 2 ===
Epoch 1/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.4536 - loss: 0.2030 - val_accuracy: 0.4470 - val_loss: 0.2737
Epoch 2/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.5315 - loss: 0.1750 - val_accuracy: 0.6034 - val_loss: 0.2540
Epoch 3/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.5686 - loss: 0.1609 - val_accuracy: 0.7136 - val_loss: 0.1764
Epoch 4/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.6040 - loss: 0.1490 - val_accuracy: 0.7019 - val_loss: 0.2054
Epoch 5/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.6401 - loss: 0.1357 - val_accuracy: 0.7156 - val_loss: 0.1882
Epoch 6/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.6611 - loss: 0.1294 - val_accuracy: 0.6954 - val_loss: 0.1772
Epoch 7/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.6861 - loss: 0.1210 - val_accuracy: 0.7256 - val_loss: 0.1545
Epoch 8/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accurac

In [14]:
process_week(3, best_params, results)


=== Processing Week 3 ===
Epoch 1/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.4835 - loss: 0.1954 - val_accuracy: 0.5455 - val_loss: 0.2555
Epoch 2/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.6267 - loss: 0.1382 - val_accuracy: 0.6346 - val_loss: 0.1979
Epoch 3/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.6962 - loss: 0.1139 - val_accuracy: 0.6658 - val_loss: 0.1584
Epoch 4/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.7478 - loss: 0.0940 - val_accuracy: 0.6059 - val_loss: 0.1412
Epoch 5/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.7721 - loss: 0.0834 - val_accuracy: 0.7346 - val_loss: 0.1122
Epoch 6/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.7990 - loss: 0.0738 - val_accuracy: 0.7004 - val_loss: 0.0908
Epoch 7/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.8085 - loss: 0.0702 - val_accuracy: 0.7892 - val_loss: 0.0571
Epoch 8/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accurac

In [15]:
process_week(4, best_params, results)


=== Processing Week 4 ===
Epoch 1/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.4985 - loss: 0.1890 - val_accuracy: 0.6969 - val_loss: 0.1393
Epoch 2/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.7791 - loss: 0.0762 - val_accuracy: 0.6194 - val_loss: 0.1129
Epoch 3/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.8578 - loss: 0.0464 - val_accuracy: 0.7553 - val_loss: 0.0486
Epoch 4/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.8881 - loss: 0.0340 - val_accuracy: 0.8259 - val_loss: 0.0473
Epoch 5/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.8997 - loss: 0.0299 - val_accuracy: 0.8401 - val_loss: 0.0319
Epoch 6/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.9185 - loss: 0.0237 - val_accuracy: 0.8675 - val_loss: 0.0296
Epoch 7/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.9170 - loss: 0.0233 - val_accuracy: 0.8755 - val_loss: 0.0277
Epoch 8/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accurac

In [16]:
# Hiển thị dữ liệu của các tuần
print("\n=== Summary Results for All Weeks ===")
for result in results:
    print(f"Week {result['week']}:")
    print(f"  Train Time: {result['train_time']:.2f} seconds")
    print(f"  Test Time: {result['test_time']:.2f} seconds")
    print(f"  Accurancy: {result['accuracy']}")
    print(f"  Precision: {result['precision']}")
    print(f"  Recall: {result['recall']}")
    print(f"  F1-Score: {result['f1_score']}")
    print(f"  Macro Precision: {result['precision_macro']}")
    print(f"  Macro Recall: {result['recall_macro']}")
    print(f"  Macro F1-Score: {result['f1_macro']}")
    print(f"  Confusion Matrix:\n{result['confusion_matrix']}")
    print("\n=== AUC-ROC ===")
    print(f"AUC Macro: {auc_macro:.4f}")
    print(f"AUC Weighted: {auc_weighted:.4f}")
    print(f"AUC per Label: {auc_per_class}")


=== Summary Results for All Weeks ===
Week 1:
  Train Time: 270.85 seconds
  Test Time: 0.38 seconds
  Accurancy: 0.6223306894447834
  Precision: [0.8173913  0.06341463 0.37195122 0.52325581 0.75491113]
  Recall: [0.25066667 0.23636364 0.59223301 0.43269231 0.80538922]
  F1-Score: [0.38367347 0.1        0.45692884 0.47368421 0.77933366]
  Macro Precision: 0.5061848207717643
  Macro Recall: 0.463468968397647
  Macro F1-Score: 0.43872403482087935
  Confusion Matrix:
[[ 94  43  32  16 190]
 [  9  13   7   0  26]
 [  6   8  61   8  20]
 [  1   9  23  45  26]
 [  5 132  41  17 807]]

=== AUC-ROC ===
AUC Macro: 0.9957
AUC Weighted: 0.9974
AUC per Label: [1.         0.99972196 0.98574342 0.99525592 0.99768956]
Week 2:
  Train Time: 272.26 seconds
  Test Time: 0.38 seconds
  Accurancy: 0.6333129957291032
  Precision: [0.56420233 0.56818182 0.27230047 0.33108108 0.78885316]
  Recall: [0.38666667 0.90909091 0.5631068  0.47115385 0.73453094]
  F1-Score: [0.45886076 0.6993007  0.36708861 0.388888